In [ ]:
!pip install gym[atari]

In [21]:
%matplotlib widget

In [3]:
import torch
from torch import nn
from torch.distributions import Categorical
from torch.optim import Adam, SGD

In [22]:
from matplotlib import pyplot as plot

In [4]:
import copy
from time import sleep

In [5]:
import numpy

In [6]:
import gym

In [7]:
class ResLinear(nn.Module):
    def __init__(self, n_in, n_out, act=nn.ReLU()):
        super(ResLinear, self).__init__()
        self.act = act
        self.linear = nn.Linear(n_in, n_out)
        self.bn = nn.BatchNorm1d(n_out)
        
        assert(n_in == n_out)
    
    def forward(self, x):
        h = self.act(self.bn(self.linear(x)))
        return h + x

In [8]:
class Player(nn.Module):
    def __init__(self, n_in=128, n_hid=100, n_out=6):
        super(Player, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, n_out))
        self.softmax = nn.Softmax()
    
    def forward(self, obs, normalized=False):
        if normalized:
            return self.softmax(self.layers(obs))
        else:
            return self.layers(obs)

In [9]:
class Value(nn.Module):
    def __init__(self, n_in=128, n_hid=100):
        super(Value, self).__init__()
        self.layers = nn.Sequential(nn.Linear(n_in, n_hid),
                                    nn.BatchNorm1d(n_hid),
                                    nn.ReLU(),
                                    ResLinear(n_hid, n_hid, nn.ReLU()),
                                    nn.Linear(n_hid, 1))
    
    def forward(self, obs):
        return self.layers(obs)

In [10]:
def copy_params(from_, to_):
    for f_, t_ in zip(from_.parameters(), to_.parameters()):
        t_.data.copy_(f_.data)
        
def avg_params(from_, to_, coeff=0.95):
    for f_, t_ in zip(from_.parameters(), to_.parameters()):
        t_.data.copy_(coeff * t_.data + (1.-coeff) * f_.data)

In [11]:
def normalize_obs(obs):
    return obs.astype('float32') / 255.

In [12]:
env = gym.make('Pong-ram-v0')

In [13]:
# collect data
def collect_one_episode(env, player, max_len=50, discount_factor=0.9, deterministic=False, rendering=False, verbose=False):
    episode = []

    observations = []

    rewards = []
    crewards = []

    actions = []
    action_probs = []

    obs = env.reset()
    
    for ml in range(max_len):
        if rendering:
            env.render()
            sleep(0.05)
            
        obs = normalize_obs(obs)

        out_probs = player(torch.from_numpy(obs[None,:]), normalized=True).squeeze()
        
        if deterministic:
            action = numpy.argmax(out_probs.data.numpy())
            if verbose:
                print(out_probs, action)
        else:
            act_dist = Categorical(out_probs)
            action = act_dist.sample().item()
        action_prob = out_probs[action].item()

        observations.append(obs)
        actions.append(action)
        action_probs.append(action_prob)

        obs, reward, done, info = env.step(action)
        if deterministic and verbose:
            print(reward, done)
        
        rewards.append(reward)

    rewards = numpy.array(rewards)

    # it's probably not the best idea to compute the discounted cumulative returns here, but well..
    for ri in range(len(rewards)):
        factors = (discount_factor ** numpy.arange(len(rewards)-ri))
        crewards.append(numpy.sum(rewards[ri:] * factors))
        
    # discard the final 10%, because it really doesn't give me a good signal due to the unbounded horizon
    # this is only for training, not for computing the total return of the episode of the given length
    discard = max_len // 10
        
    return observations[:-discard], crewards[:-discard], actions[:-discard], action_probs[:-discard], rewards.sum()

In [14]:
# simple implementation of FIFO-based replay buffer
class Buffer:
    def __init__(self, max_items=10000):
        self.max_items = max_items
        self.buffer = []
        
    def add(self, observations, crewards, actions, action_probs):
        new_n = len(observations)
        old_n = len(self.buffer)
        if new_n + old_n > self.max_items:
            del self.buffer[:new_n]
        for o, c, a, p in zip(observations, crewards, actions, action_probs):
            self.buffer.append((o, c, a, p))
            
    def sample(self, n=100):
        idxs = numpy.random.choice(len(self.buffer),n)
        return [self.buffer[ii] for ii in idxs]

In [15]:
# create a policy
player = Player(n_in=128, n_hid=32, n_out=6)

In [16]:
# create a value estimator
value = Value(n_in=128, n_hid=32)

In [17]:
# initialize optimizers
opt_player = Adam(player.parameters(), lr=0.0001)
opt_value = Adam(value.parameters(), lr=0.0001)

In [18]:
# initialize replay buffer
replay_buffer = Buffer(max_items=10000)

In [19]:
n_iter = 10000
init_collect = 10
n_collect = 1
n_value = 10
n_policy = 10
disp_iter = 1
val_iter = 1

max_len = 1000
batch_size = 500

ent_coeff = 0. #0.001
discount_factor = .95

value_loss = -numpy.Inf
ret = -numpy.Inf
entropy = -numpy.Inf

return_history = []

for ni in range(n_iter):
    player.eval()

    if numpy.mod(ni, val_iter) == 0:
        _, _, _, _, ret_ = collect_one_episode(env, player, max_len=max_len, deterministic=True)
        print('Valid run', ret_)
        return_history.append(ret_)

    # collect some episodes using the current policy
    # and push (obs,a,r,p(a)) tuples to the replay buffer.
    nc = n_collect
    if ni == 0:
        nc = init_collect
    for ci in range(nc):
        o_, c_, a_, ap_, ret_ = collect_one_episode(env, player, max_len=max_len, discount_factor=discount_factor)
        replay_buffer.add(o_, c_, a_, ap_)
        if ret == -numpy.Inf:
            ret = ret_
        else:
            ret = 0.9 * ret + 0.1 * ret_
    
    player.train()
        
    # fit a value function
    for vi in range(n_value):
        opt_player.zero_grad()
        opt_value.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        batch_x = torch.from_numpy(numpy.stack([ex[0] for ex in batch]).astype('float32'))
        batch_y = torch.from_numpy(numpy.stack([ex[1] for ex in batch]).astype('float32'))
        pred_y = value(batch_x).squeeze()
        loss_ = ((batch_y - pred_y) ** 2)
        
        batch_a = torch.from_numpy(numpy.stack([ex[2] for ex in batch]).astype('float32')[:,None])
        batch_pi = player(batch_x, normalized=True)
        batch_q = torch.from_numpy(numpy.stack([ex[3] for ex in batch]).astype('float32'))
        logp = torch.log(batch_pi.gather(1, batch_a.long()))

        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss_
        
        loss = loss.mean()
        
        loss.backward()
        opt_value.step()
        
    if value_loss < 0.:
        value_loss = loss_.mean().item()
    else:
        value_loss = 0.9 * value_loss + 0.1 * loss_.mean().item()
    
    if numpy.mod(ni, disp_iter) == 0:
        print('# plays', (ni+1) * n_collect, 'return', ret, 'value_loss', value_loss, 'entropy', -entropy)
    
    # fit a policy
    for pi in range(n_policy):
        opt_player.zero_grad()
        opt_value.zero_grad()
        
        batch = replay_buffer.sample(batch_size)
        
        batch_x = torch.from_numpy(numpy.stack([ex[0] for ex in batch]).astype('float32'))
        batch_r = torch.from_numpy(numpy.stack([ex[1] for ex in batch]).astype('float32')[:,None])
        batch_v = value(batch_x)
        batch_a = torch.from_numpy(numpy.stack([ex[2] for ex in batch]).astype('float32')[:,None])
        batch_q = torch.from_numpy(numpy.stack([ex[3] for ex in batch]).astype('float32'))

        batch_pi = player(batch_x, normalized=True)
        
        logp = torch.log(batch_pi.gather(1, batch_a.long()))
        
        # advantage
        adv = batch_r - batch_v.clone().detach()
        
        loss = -(adv * logp)
        
        # (clipped) importance weight: 
        # because the policy may have changed since the tuple was collected.
        iw = torch.exp((logp.clone().detach() - torch.log(batch_q)).clamp(max=0.))
    
        loss = iw * loss
        
        # entropy regularization: though, it doesn't look necessary in this specific case.
        ent = (batch_pi * torch.log(batch_pi)).sum(1)
        
        if entropy == -numpy.Inf:
            entropy = ent.mean().item()
        else:
            entropy = 0.9 * entropy + 0.1 * ent.mean().item()
        
        loss = (loss + ent_coeff * ent).mean()
        
        loss.backward()
        opt_player.step()

/Users/kyunghyuncho/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


Valid run -20.0
# plays 1 return -16.542701650999998 value_loss 0.26948466897010803 entropy inf
Valid run -20.0
# plays 2 return -16.8884314859 value_loss 0.26769939363002776 entropy 1.7384350076243655
Valid run -20.0
# plays 3 return -16.59958833731 value_loss 0.26605464190244676 entropy 1.7384942054500834
Valid run -20.0
# plays 4 return -16.539629503579 value_loss 0.2576955691277981 entropy 1.738444260076567
Valid run -20.0
# plays 5 return -16.785666553221102 value_loss 0.2503564210504294 entropy 1.7377413517327507
Valid run -20.0
# plays 6 return -16.707099897898992 value_loss 0.24135280267328027 entropy 1.7375544968376355
Valid run -20.0
# plays 7 return -16.836389908109094 value_loss 0.23014565511879329 entropy 1.7375649876272936
Valid run -20.0
# plays 8 return -16.852750917298184 value_loss 0.22176235649887027 entropy 1.737181644152176
Valid run -20.0
# plays 9 return -16.867475825568366 value_loss 0.21341569632801385 entropy 1.7371947061278707
Valid run -20.0
# plays 10 retur

KeyboardInterrupt: 

In [25]:
plot.figure()

plot.plot(return_history)
plot.grid(True)
plot.xlabel('# of plays x {}'.format(n_collect))
plot.ylabel('Return over the episode of length {}'.format(max_len))

plot.show()

FigureCanvasNbAgg()

In [26]:
# let the final policy play the pong longer
player.eval()
_, _, _, _, ret_ = collect_one_episode(env, player, max_len=1000000, deterministic=True, rendering=True)

/Users/kyunghyuncho/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  del sys.path[0]


KeyboardInterrupt: 